In [1]:
import threading, logging, time
import multiprocessing

from kafka import KafkaConsumer, KafkaProducer

kafkaUrl = "smartaqnet-dev.teco.edu"
#kafkaUrl = "smartaqnet.teco.edu"
class Producer(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)
        self.stop_event = threading.Event()
        
    def stop(self):
        self.stop_event.set()

    def run(self):
        producer = KafkaProducer(bootstrap_servers=kafkaUrl+':9092')

        while not self.stop_event.is_set():
            producer.send('my-topic', b"test")
            producer.send('my-topic', b"\xc2Hola, mundo!")
            time.sleep(1)

        producer.close()

class Consumer(multiprocessing.Process):
    def __init__(self,topic):
        multiprocessing.Process.__init__(self)
        self.stop_event = multiprocessing.Event()
        self.consumer = KafkaConsumer(bootstrap_servers= kafkaUrl + ':9092',
                         auto_offset_reset='earliest',
                         consumer_timeout_ms=1000)
        self.consumer.subscribe(topic)
        
    def stop(self):
        self.stop_event.set()
        
    def run(self):
        

        while not self.stop_event.is_set():
            for message in self.consumer:
                print(message)
                if self.stop_event.is_set():
                    break

        self.consumer.close()

In [4]:
tasks = [
        #Producer(),
        Consumer(['WG_Measurements'])
]
for t in tasks:
        t.start()

ConsumerRecord(topic=u'WG_Measurements', partition=0, offset=0, timestamp=1526303376712, timestamp_type=0, key='edu.teco.wang-IBADENWR82-Datastreams-AirTemperature-IBADENWR82', value="Row(ID=u'IBADENWR82', Time=u'00:00:00', Latitude=u'48.441', Longitude=u'9.257', AirTemperature=u'1.8')", checksum=None, serialized_key_size=62, serialized_value_size=102)
ConsumerRecord(topic=u'WG_Measurements', partition=0, offset=1, timestamp=1526303378149, timestamp_type=0, key='edu.teco.wang-IBADENWR82-Datastreams-AirTemperature-IBADENWR82', value="Row(ID=u'IBADENWR82', Time=u'00:50:00', Latitude=u'48.441', Longitude=u'9.257', AirTemperature=u'1.8')", checksum=None, serialized_key_size=62, serialized_value_size=102)


In [3]:
for t in tasks:
        t.stop()